In [53]:
import pandas as pd
from utz import *

import duckdb
dql = duckdb.sql

from njdot import NJDOT_DIR
from nj_crashes.paths import PUBLIC_DIR, WWW_DOT

In [4]:
%%time
dql(f"IMPORT DATABASE '{WWW_DOT}/crashes.ddb'")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 11.3 s, sys: 1.01 s, total: 12.3 s
Wall time: 3.83 s


In [278]:
%%time
dot = dql("""
SELECT *
FROM crashes
WHERE strftime(dt, '%Y') == '2021'
ORDER BY dt
""").df().set_index('id')

dotr = dot.reset_index().rename(columns={
    'Total Killed': 'tk',
    'Total Injured': 'ti',
    'Pedestrians Killed': 'pk',
    'Route': 'rt',
})
dotr = dotr.rename(columns={ 'id': 'dot_id', })
dotr['ccmc'] = dotr.apply(lambda r: '%02d%02d' % (r.cc, r.mc), axis=1)
dotr

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 4.33 s, sys: 1.97 s, total: 6.31 s
Wall time: 5.06 s


,dot_id,year,cc,mc,case,cn,mn,pdc,pdn,station,tk,ti,pk,Pedestrians Injured,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,Total Vehicles Involved,Crash Location,Location Direction,rt,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event,ocn,reason,ilon,ilat,icn,ccmc
0,5850346,2021,1,1,I-2021-000001,Atlantic,Absecon City,01,Absecon City Pd,,0,0,0,0,p,B,False,False,11,1,US 30,E,30,,00000030__,50.80,02,,02,01,06,01,01,01,175,FE,,,T,HADDON AVE,NB,45,,NaN,NaN,N,,846,2021-01-01 00:00:00,03,04,56,None,None,-74.516528,39.435744,Atlantic,0101
1,5884854,2021,3,18,C020-2021-00001A,Burlington,Mansfield Twp,02,New Jersey State Police,BORDENTOWN - SQ,0,2,0,0,i,B,True,False,11,1,I-295,S,295,,00000295__,54.60,01,,02,02,06,02,01,01,,,,,,,,65,,40.11420,-74.72824,N,,8341,2021-01-01 00:00:00,03,04,46,Burlington,None,-74.735220,40.109480,Burlington,0318
2,5959766,2021,10,16,C040-2021-00001A,Hunterdon,Kingwood Twp,02,New Jersey State Police,KINGWOOD - SQUA,0,0,0,0,p,B,False,False,12,1,LOCKTOWN SCHOOL RD,S,,,,NaN,07,,02,01,05,01,05,01,300,FE,S,POINT BREEZE RD.,,,,35,35,40.50287,-74.97630,N,,8356,2021-01-01 00:00:00,01,04,24,Hunterdon,No MP,NaN,NaN,None,1016
3,6058030,2021,20,4,21-000005,Union,Elizabeth City,01,Elizabeth Pd,,0,0,0,0,p,I,False,False,01,2,US 1,N,1,,00000001__,43.39,02,,02,01,06,01,01,01,,AT,,CR 623 / S BROAD ST,,,,25,40,NaN,NaN,N,,867,2021-01-01 00:06:00,01,04,26,None,None,-74.215945,40.652137,Union,2004
4,6039196,2021,16,8,I-2021-000005,Passaic,Paterson City,01,Paterson Pd,PATERSON PD 112,0,0,0,0,p,I,False,False,06,2,N 1 ST ST,N,,,,NaN,07,,02,01,06,01,04,01,,AT,,,,,,,,NaN,NaN,N,,4984,2021-01-01 00:07:00,01,04,28,None,No MP,NaN,NaN,None,1608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226953,5950352,2021,9,6,21-262666,Hudson,Jersey City,01,Jersey City Pd,SOUTH DISTRICT,0,0,0,0,p,I,False,False,01,2,ROUTE 501,N,501,,00000501__,28.00,05,,02,01,06,01,04,,,AT,N,WADE ST,,,,25,25,NaN,NaN,N,,3357,2021-12-31 23:42:00,01,04,26,None,None,-74.091196,40.702851,Hudson,0906
226954,5950351,2021,9,6,21-262657,Hudson,Jersey City,01,Jersey City Pd,NORTH,0,0,0,0,p,I,False,False,03,2,SUMMIT AVE,N,,,09061700__,0.59,07,,02,01,06,01,04,01,,AT,,PATERSON ST,,,,25,25,NaN,NaN,N,,3337,2021-12-31 23:45:00,01,04,26,None,None,-74.048755,40.751314,Hudson,0906
226955,5968054,2021,11,11,22000001,Mercer,Trenton City,01,Trenton Pd,TRENTON,0,0,0,0,p,I,False,False,05,2,US 206,N,206,,00000206__,43.63,07,,02,01,07,01,05,01,,AT,,SOUTHARD ST,,,,25,25,NaN,NaN,N,,6632,2021-12-31 23:48:00,01,04,26,None,None,-74.758447,40.229827,Mercer,1111
226956,5903897,2021,5,4,210687830,Cape May,Dennis Twp,02,New Jersey State Police,A050,0,0,0,0,p,B,False,False,12,1,NJ 47,S,47,,00000047__,21.50,02,,02,02,05,02,05,01,,,,,,,,50,,39.20307,-74.88470,N,,6314,2021-12-31 23:50:00,01,04,24,Cape May,None,-74.875787,39.205192,Cape May,0504


## fsck DOT muni codes/names

In [361]:
dot_mn_hist = dotr[['cc', 'mn', 'mc']].value_counts().rename('count').sort_index().reset_index(level=2)
dot_mn_hist2 = dot_mn_hist.groupby(lambda _:_)['mc'].count()
assert dot_mn_hist2[dot_mn_hist2 > 1].empty

dot_ccmn2mc = dot_mn_hist.mc
dot_ccmn2mc

cc  mn               
1   Absecon City          1
    Atlantic City         2
    Brigantine City       3
    Buena Boro            4
    Buena Vista Twp       5
                         ..
21  Phillipsburg Town    19
    Pohatcong Twp        20
    Washington Boro      21
    Washington Twp       22
    White Twp            23
Name: mc, Length: 556, dtype: int64

In [362]:
ccmn2mc = dot_ccmc2mn.to_dict()
ccmn2mc

{(1, 'Absecon City'): 1,
 (1, 'Atlantic City'): 2,
 (1, 'Brigantine City'): 3,
 (1, 'Buena Boro'): 4,
 (1, 'Buena Vista Twp'): 5,
 (1, 'Corbin City'): 6,
 (1, 'Egg Harbor City'): 7,
 (1, 'Egg Harbor Twp'): 8,
 (1, 'Estell Manor City'): 9,
 (1, 'Folsom Boro'): 10,
 (1, 'Galloway Twp'): 11,
 (1, 'Hamilton Twp'): 12,
 (1, 'Hammonton Town'): 13,
 (1, 'Linwood City'): 14,
 (1, 'Longport Boro'): 15,
 (1, 'Margate City'): 16,
 (1, 'Mullica Twp'): 17,
 (1, 'Northfield City'): 18,
 (1, 'Pleasantville City'): 19,
 (1, 'Port Republic City'): 20,
 (1, 'Somers Point City'): 21,
 (1, 'Ventnor City'): 22,
 (1, 'Weymouth Twp'): 23,
 (2, 'Allendale Boro'): 1,
 (2, 'Alpine Boro'): 2,
 (2, 'Bergenfield Boro'): 3,
 (2, 'Bogota Boro'): 4,
 (2, 'Carlstadt Boro'): 5,
 (2, 'Cliffside Park Boro'): 6,
 (2, 'Closter Boro'): 7,
 (2, 'Demarest Boro'): 9,
 (2, 'Dumont Boro'): 10,
 (2, 'East Rutherford Boro'): 12,
 (2, 'Edgewater Boro'): 13,
 (2, 'Elmwood Park Boro'): 11,
 (2, 'Emerson Boro'): 14,
 (2, 'Englewood Ci

In [380]:
sp_all = pd.read_sql_table("crashes", "sqlite:///../nj_crashes.db")
sp = sp_all[sp_all.dt.dt.year == 2021].copy()
sp['cc'] = sp.CCODE.astype(int)
#sp['mc'] = sp.MCODE.str[2:].astype(int)
sp['mn'] = sp.MNAME.copy()
replace_mns = {
    ' T(?:w(?:sp?)?)?$': ' Twp',
    ' Town$': ' Twp',
    ' Bor?$': ' Boro',
    ' Cit$': ' City',
    ' To$': ' Town',
    ' Hil$': ' Hills',
    'Ridgefield Park Twp': 'Ridgefield Park Village',
    'Bloomfield Boro': 'Bloomfield Twp',
    'Hazlet Boro': 'Hazlet Twp',
    'Milford Boro': 'Milford Twp',
    'Trenton': 'Trenton City',
    'Dover Twp': 'Dover Town',
    'Hammonton Twp': 'Hammonton Town',
    'Kearny Twp': 'Kearny Town',
    'Morristown Twp': 'Morristown Town',
    'West New York Twp': 'West New York Town',
    'Westfield Twp': 'Westfield Town',
    'Toms River': 'Toms River Twp',
}
for before, after in replace_mns.items():
    sp['mn'] = sp['mn'].str.replace(before, after, regex=True)

sp['mc'] = sp[['cc', 'mn']].apply(lambda r: ccmn2mc[(r.cc, r.mn)], axis=1)
sp['ccmc'] = sp.MCODE
sp['ACCID'] = sp['ACCID'].astype(int)
sp = sxs(sp, sp.LOCATION.str.extract(r'^(?P<street>.*?)(?: MP ?(?P<mp>\d*(?:\.\d+)?))?$')).set_index('ACCID')
sp['mp'] = sp['mp'].astype(float)
sp = sp.rename(columns={
    'FATALITIES': 'tk',
    'INJURIES': 'ti',
    'FATAL_T': 'pk',
    'HIGHWAY': 'rt',
    'CNAME': 'cn',
})
sp = sp.drop(columns=[ 'FATAL_P', 'FATAL_B', 'FATAL_D', 'CCODE', 'MCODE', ])
for k in [ 'tk', 'ti', 'pk' ]:
    sp[k] = sp[k].fillna(0).astype(int)
sp['rt'] = sp['rt'].apply(lambda rt: '' if rt is None else str(rt))
sp = sp.sort_index()
sp

,cn,MNAME,rt,LOCATION,tk,ti,STREET,pk,dt,cc,mn,mc,ccmc,street,mp
ACCID,,,,,,,,,,,,,,,
10626,Union,Union Twsp,78,Interstate 78 MP 50.8,1,0,None,0,2021-01-01 09:06:00,20,Union Twp,19,2019,Interstate 78,50.80
10627,Mercer,West Windsor Twp,1,State Highway 1 MP 11.83,1,1,None,0,2021-01-02 09:07:00,11,West Windsor Twp,13,1113,State Highway 1,11.83
10628,Gloucester,Franklin Twsp,555,County 555 MP 24.9,1,2,None,0,2021-01-03 04:19:00,8,Franklin Twp,5,0805,County 555,24.90
10630,Ocean,Barnegat Twsp,444,Garden State Parkway MP 68.1,1,0,None,0,2021-01-01 07:45:00,15,Barnegat Twp,33,1533,Garden State Parkway,68.10
10632,Middlesex,Woodbridge Twsp,616,County 616 MP .29,1,2,None,1,2021-01-04 13:35:00,12,Woodbridge Twp,25,1225,County 616,0.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11581,Gloucester,Washington Twsp,651,County 651 MP 2.9,1,0,None,1,2021-12-09 23:44:00,8,Washington Twp,18,0818,County 651,2.90
11724,Middlesex,Highland Park Boro,27,State Highway 27 MP 17.3,1,0,None,0,2021-10-17 08:41:00,12,Highland Park Boro,7,1207,State Highway 27,17.30
11730,Mercer,Trenton,,Elmer St,1,0,Elmer St,0,2021-03-23 12:49:00,11,Trenton City,11,1111,Elmer St,NaN


# fsck muni codes/names

In [381]:
sp_mn_hist = sp[['cc', 'mn', 'mc']].value_counts().rename('count').sort_index().reset_index(level=2)
sp_mn_hist2 = sp_mn_hist.groupby(lambda _:_)['mc'].count()
assert sp_mn_hist2[sp_mn_hist2 > 1].empty

sp_ccmn2mc = sp_mn_hist.mc
sp_ccmn2mc

cc  mn               
1   Absecon City          1
    Atlantic City         2
    Brigantine City       3
    Buena Vista Twp       5
    Egg Harbor Twp        8
                         ..
21  Knowlton Twp         13
    Oxford Twp           17
    Phillipsburg Town    19
    Washington Twp       22
    White Twp            23
Name: mc, Length: 244, dtype: int64

In [382]:
sp_mns = sp_ccmn2mc.reset_index(level=1).mn.groupby(lambda _:_).apply(set).to_dict()
dot_mns = dot_ccmn2mc.reset_index(level=1).mn.groupby(lambda _:_).apply(set).to_dict()

In [383]:
for cc, sps in sp_mns.items():
    dots = dot_mns[cc]
    sp_only = list(sorted(sps.difference(dots)))
    dot_only = list(sorted(dots.difference(sps)))
    if sp_only:
        err(f'cc {cc}:')
        err(f'\tsp_only: {", ".join(sp_only)}')
        err(f'\tdot_only: {", ".join(dot_only)}')
        err('')

In [384]:
sp_mns[8], dot_mns[8]

({'Clayton Boro',
  'Deptford Twp',
  'East Greenwich Twp',
  'Elk Twp',
  'Franklin Twp',
  'Harrison Twp',
  'Logan Twp',
  'Mantua Twp',
  'Monroe Twp',
  'South Harrison Twp',
  'Washington Twp',
  'West Deptford Twp',
  'Westville Boro'},
 {'Clayton Boro',
  'Deptford Twp',
  'East Greenwich Twp',
  'Elk Twp',
  'Franklin Twp',
  'Glassboro Boro',
  'Greenwich Twp',
  'Harrison Twp',
  'Logan Twp',
  'Mantua Twp',
  'Monroe Twp',
  'National Park Boro',
  'Newfield Boro',
  'Paulsboro Boro',
  'Pitman Boro',
  'South Harrison Twp',
  'Swedesboro Boro',
  'Washington Twp',
  'West Deptford Twp',
  'Westville Boro',
  'Woodbury City',
  'Woodbury Heights Boro',
  'Woolwich Twp'})

In [58]:
pd.crosstab(sp.STREET.isna(), sp.HIGHWAY.isna())

HIGHWAY,False,True
STREET,,
False,0,122
True,545,0


In [89]:
dotr.tk.sum(), sp.tk.sum()

(721, 697)

In [385]:
def match_crash(c, keys, dot_ids, dot_dict):
    k = tuple(c[keys])
    if k in dot_dict:
        matches = dot_ids.loc[k]
        if len(matches) == 1:
            dot_id = matches[0]
            c['dot_id'] = dot_id
        elif len(matches):
            err(f"ACCID {c.ACCID} matches {len(matches)} DOT crashes")
        return c
    else:
        return c

def match(sp, keys):
    dot_ids = dotr.set_index(keys).dot_id.sort_index()
    dot_dict = dot_ids.to_dict()
    nxt = (
        sp
        .reset_index()
        .apply(match_crash, keys=keys, dot_ids=dot_ids, dot_dict=dot_dict, axis=1)
        .set_index('ACCID')
        .sort_index()
    )
    mask = nxt.dot_id.isna()
    mask_hist = mask.value_counts()
    num_missing = mask_hist.loc[True]
    num_found = mask_hist.loc[False]
    print(f'Found {num_found}, missing {num_missing}')
    missing = sp[mask].copy()
    found = nxt[~mask].copy().astype({ 'dot_id': int })
    return found, missing

In [386]:
%%time
sp1, sm1 = match(sp, ['dt', 'cc', 'mc', 'tk'])
sp1

Found 575, missing 92
CPU times: user 614 ms, sys: 206 ms, total: 820 ms
Wall time: 851 ms


,LOCATION,MNAME,STREET,cc,ccmc,cn,dot_id,dt,mc,mn,mp,pk,rt,street,ti,tk
ACCID,,,,,,,,,,,,,,,,
10626,Interstate 78 MP 50.8,Union Twsp,None,20,2019,Union,6071677,2021-01-01 09:06:00,19,Union Twp,50.80,0,78,Interstate 78,0,1
10627,State Highway 1 MP 11.83,West Windsor Twp,None,11,1113,Mercer,5968575,2021-01-02 09:07:00,13,West Windsor Twp,11.83,0,1,State Highway 1,1,1
10628,County 555 MP 24.9,Franklin Twsp,None,8,0805,Gloucester,5936843,2021-01-03 04:19:00,5,Franklin Twp,24.90,0,555,County 555,2,1
10632,County 616 MP .29,Woodbridge Twsp,None,12,1225,Middlesex,5987570,2021-01-04 13:35:00,25,Woodbridge Twp,0.29,1,616,County 616,2,1
10633,Fairview St,Camden City,Fairview St,4,0408,Camden,5892434,2021-01-05 02:03:00,8,Camden City,NaN,0,,Fairview St,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11573,State Highway 1 MP 46.3,Newark City,None,7,0714,Essex,5924105,2021-05-01 19:26:00,14,Newark City,46.30,0,1,State Highway 1,2,1
11581,County 651 MP 2.9,Washington Twsp,None,8,0818,Gloucester,5940813,2021-12-09 23:44:00,18,Washington Twp,2.90,1,651,County 651,0,1
11724,State Highway 27 MP 17.3,Highland Park Boro,None,12,1207,Middlesex,5975837,2021-10-17 08:41:00,7,Highland Park Boro,17.30,0,27,State Highway 27,0,1


In [387]:
sm1

,cn,MNAME,rt,LOCATION,tk,ti,STREET,pk,dt,cc,mn,mc,ccmc,street,mp
ACCID,,,,,,,,,,,,,,,
10630,Ocean,Barnegat Twsp,444,Garden State Parkway MP 68.1,1,0,None,0,2021-01-01 07:45:00,15,Barnegat Twp,33,1533,Garden State Parkway,68.10
10668,Gloucester,Logan Twsp,295,Interstate 295 MP 15.1,1,0,None,0,2021-01-19 12:32:00,8,Logan Twp,9,0809,Interstate 295,15.10
10694,Camden,Gloucester City,130,State Highway 130 MP 26.9,1,2,None,0,2021-01-25 05:03:00,4,Gloucester City,14,0414,State Highway 130,26.90
10721,Passaic,Paterson City,,East 18th St,1,0,East 18th St,1,2021-02-13 22:39:00,16,Paterson City,8,1608,East 18th St,NaN
10733,Hudson,Jersey City,,Christopher Columbus Dr,1,1,Christopher Columbus Dr,0,2021-02-28 03:28:00,9,Jersey City,6,0906,Christopher Columbus Dr,NaN
10755,Middlesex,Sayreville Boro,35,State Highway 35 MP 47.43,1,0,None,0,2021-02-28 15:21:00,12,Sayreville Boro,19,1219,State Highway 35,47.43
10764,Hudson,Jersey City,,John F. Kennedey Blvd,1,0,John F. Kennedey Blvd,0,2021-03-10 12:16:00,9,Jersey City,6,0906,John F. Kennedey Blvd,NaN
10778,Passaic,Paterson City,20,State Highway 20 MP 1.3,1,3,None,0,2021-02-24 04:51:00,16,Paterson City,8,1608,State Highway 20,1.30
10785,Ocean,Toms River,37,State Highway 37 MP 11.95,1,1,None,0,2021-03-15 11:02:00,15,Toms River Twp,7,1507,State Highway 37,11.95


In [370]:
sp2, sm2 = match(sm1, ['dt', 'cc', 'mc'])
sp2

Found 12, missing 80


,LOCATION,MNAME,STREET,cc,ccmc,cn,dot_id,dt,mc,mn,mp,pk,rt,street,ti,tk
ACCID,,,,,,,,,,,,,,,,
10630,Garden State Parkway MP 68.1,Barnegat Twsp,None,15,1533,Ocean,6031342,2021-01-01 07:45:00,33,Barnegat Twp,68.10,0,444,Garden State Parkway,0,1
10721,East 18th St,Paterson City,East 18th St,16,1608,Passaic,6039704,2021-02-13 22:39:00,8,Paterson City,NaN,1,,East 18th St,0,1
10778,State Highway 20 MP 1.3,Paterson City,None,16,1608,Passaic,6039774,2021-02-24 04:51:00,8,Paterson City,1.30,0,20,State Highway 20,3,1
10785,State Highway 37 MP 11.95,Toms River,None,15,1507,Ocean,6019832,2021-03-15 11:02:00,7,Toms River Twp,11.95,0,37,State Highway 37,1,1
11034,County 673 MP 4.5,Vineland City,None,6,0614,Cumberland,5908836,2021-07-09 23:22:00,14,Vineland City,4.50,1,673,County 673,0,1
11230,County 614 MP .9,Hamilton Twsp,None,1,0112,Atlantic,5854250,2021-09-25 14:02:00,12,Hamilton Twp,0.90,0,614,County 614,0,1
11234,State Highway 140 MP .3,Carneys Point Twsp,None,17,1713,Salem,6045864,2021-09-19 12:02:00,13,Carneys Point Twp,0.30,0,140,State Highway 140,1,1
11309,State Highway 139 (Upper (State Route 139 Only...,Jersey City,None,9,0906,Hudson,5950657,2021-10-22 07:10:00,6,Jersey City,0.58,1,139,State Highway 139 (Upper (State Route 139 Only)),0,1
11340,County 515 MP 18.2,Vernon Twsp,None,19,1922,Sussex,6056263,2021-10-30 17:14:00,22,Vernon Twp,18.20,1,515,County 515,0,1


In [403]:
m2 = sp2[['dot_id', 'dt', 'cc', 'mc', 'tk']].merge(dotr[['dot_id', 'tk', 'severity']], on='dot_id', suffixes=['_sp','_dot'])
m2

,dot_id,dt,cc,mc,tk_sp,tk_dot,severity
0,6031342,2021-01-01 07:45:00,15,33,1,2,f
1,6039704,2021-02-13 22:39:00,16,8,1,0,i
2,6039774,2021-02-24 04:51:00,16,8,1,0,i
3,6019832,2021-03-15 11:02:00,15,7,1,0,i
4,5908836,2021-07-09 23:22:00,6,14,1,0,i
5,5854250,2021-09-25 14:02:00,1,12,1,0,i
6,6045864,2021-09-19 12:02:00,17,13,1,0,i
7,5950657,2021-10-22 07:10:00,9,6,1,0,i
8,6056263,2021-10-30 17:14:00,19,22,1,0,i
9,6001736,2021-11-03 13:56:00,13,34,1,0,i


In [388]:
sp3, sm3 = match(sm2, ['dt', 'cc'])
sp3

Found 16, missing 64


,LOCATION,MNAME,STREET,cc,ccmc,cn,dot_id,dt,mc,mn,mp,pk,rt,street,ti,tk
ACCID,,,,,,,,,,,,,,,,
10802,Interstate 295 MP 28.4,Bellmawr Boro,None,4,0404,Camden,5899049,2021-03-31 15:20:00,4,Bellmawr Boro,28.40,0,295,Interstate 295,0,1
10827,Interstate 295 MP 21.2,Deptford Twsp,None,8,0802,Gloucester,5941400,2021-04-13 14:51:00,2,Deptford Twp,21.20,0,295,Interstate 295,2,1
10887,County 518 MP 10.5,Hopewell Twsp,None,11,1106,Mercer,5964038,2021-05-13 16:11:00,6,Hopewell Twp,10.50,0,518,County 518,0,1
10905,Interstate 295 MP 22.7,Deptford Twsp,None,8,0802,Gloucester,5941416,2021-05-19 06:05:00,2,Deptford Twp,22.70,1,295,Interstate 295,0,1
10910,Interstate 76 MP .4,Gloucester City,None,4,0414,Camden,5891996,2021-05-22 22:17:00,14,Gloucester City,0.40,0,76,Interstate 76,0,1
10935,County 551 MP 1.51,Pennsville Twsp,None,17,1705,Salem,6045969,2021-06-07 21:33:00,8,Pennsville Twp,1.51,0,551,County 551,0,1
10936,County 508 MP 7.95,West Orange Town,None,7,0722,Essex,5913249,2021-06-07 22:20:00,22,West Orange Twp,7.95,1,508,County 508,0,1
10940,State Highway 168 MP 10.65,Woodlynne Boro,None,4,0437,Camden,5892603,2021-06-09 22:57:00,37,Woodlynne Boro,10.65,1,168,State Highway 168,0,1
11007,State Highway 31 MP 43.25,Washington Twsp,None,21,2122,Warren,6076973,2021-07-12 22:59:00,22,Washington Twp,43.25,1,31,State Highway 31,0,1


In [400]:
keys = ['dot_id', 'cc', 'mc', 'cn', 'mn']
m3 = sp3[keys].merge(dotr[keys], on='dot_id', validate='one_to_one', suffixes=['_sp', '_dot'], )
assert (m3.cc_sp == m3.cc_dot).all()
assert (m3.cn_sp == m3.cn_dot).all()
m3[[ f'{k}_{d}' for k in [ 'mc', 'mn' ] for d in [ 'sp', 'dot' ] ]]

,mc_sp,mc_dot,mn_sp,mn_dot
0,4,18,Bellmawr Boro,Haddon Heights Boro
1,2,20,Deptford Twp,West Deptford Twp
2,6,5,Hopewell Twp,Hopewell Boro
3,2,20,Deptford Twp,West Deptford Twp
4,14,4,Gloucester City,Bellmawr Boro
5,8,13,Pennsville Twp,Carneys Point Twp
6,22,5,West Orange Twp,East Orange City
7,37,8,Woodlynne Boro,Camden City
8,22,21,Washington Twp,Washington Boro
9,18,12,Manchester Twp,Lacey Twp


In [ ]:
sp3

In [396]:
m3[m3.mc_sp == m3.mc_dot]

,dot_id,cc_sp,mc_sp,cn_sp,mn_sp,cc_dot,mc_dot,cn_dot,mn_dot


In [389]:
dot.loc[5899049]

year                                                                           2021
cc                                                                                4
mc                                                                               18
case                                                               A310-2021-00656A
cn                                                                           Camden
mn                                                              Haddon Heights Boro
pdc                                                                              02
pdn                                                         New Jersey State Police
station                                                             BELLMAWR - SQUA
Total Killed                                                                      1
Total Injured                                                                     0
Pedestrians Killed                                                          

In [373]:
sp3[sp3.MNAME == sp3.mn]

,LOCATION,MNAME,STREET,cc,ccmc,cn,dot_id,dt,mc,mn,mp,pk,rt,street,ti,tk
ACCID,,,,,,,,,,,,,,,,
10802,Interstate 295 MP 28.4,Bellmawr Boro,None,4,0404,Camden,5899049,2021-03-31 15:20:00,4,Bellmawr Boro,28.40,0,295,Interstate 295,0,1
10910,Interstate 76 MP .4,Gloucester City,None,4,0414,Camden,5891996,2021-05-22 22:17:00,14,Gloucester City,0.40,0,76,Interstate 76,0,1
10940,State Highway 168 MP 10.65,Woodlynne Boro,None,4,0437,Camden,5892603,2021-06-09 22:57:00,37,Woodlynne Boro,10.65,1,168,State Highway 168,0,1
11435,State Highway 22 MP 38.5,Bound Brook Boro,None,18,1804,Somerset,6047401,2021-05-26 11:18:00,4,Bound Brook Boro,38.50,0,22,State Highway 22,0,1


In [374]:
sp3[sp3.MNAME != sp3.mn]

,LOCATION,MNAME,STREET,cc,ccmc,cn,dot_id,dt,mc,mn,mp,pk,rt,street,ti,tk
ACCID,,,,,,,,,,,,,,,,
10827,Interstate 295 MP 21.2,Deptford Twsp,None,8,0802,Gloucester,5941400,2021-04-13 14:51:00,2,Deptford Twp,21.20,0,295,Interstate 295,2,1
10887,County 518 MP 10.5,Hopewell Twsp,None,11,1106,Mercer,5964038,2021-05-13 16:11:00,6,Hopewell Twp,10.50,0,518,County 518,0,1
10905,Interstate 295 MP 22.7,Deptford Twsp,None,8,0802,Gloucester,5941416,2021-05-19 06:05:00,2,Deptford Twp,22.70,1,295,Interstate 295,0,1
10935,County 551 MP 1.51,Pennsville Twsp,None,17,1705,Salem,6045969,2021-06-07 21:33:00,8,Pennsville Twp,1.51,0,551,County 551,0,1
10936,County 508 MP 7.95,West Orange Town,None,7,0722,Essex,5913249,2021-06-07 22:20:00,22,West Orange Twp,7.95,1,508,County 508,0,1
11007,State Highway 31 MP 43.25,Washington Twsp,None,21,2122,Warren,6076973,2021-07-12 22:59:00,22,Washington Twp,43.25,1,31,State Highway 31,0,1
11189,County 614 MP 1.05,Manchester Twsp,None,15,1518,Ocean,6023982,2021-07-22 12:57:00,18,Manchester Twp,1.05,0,614,County 614,1,1
11190,Javes Rd,Holland Twsp,Javes Rd,10,1015,Hunterdon,5960009,2021-09-02 07:22:00,15,Holland Twp,NaN,0,,Javes Rd,0,1
11192,Interstate 295 MP 17,East Greenwich Twsp,None,8,0803,Gloucester,5937981,2021-07-08 14:10:00,3,East Greenwich Twp,17.00,0,295,Interstate 295,1,1


In [372]:
(sp3.MNAME == sp3.mn).value_counts()

False    12
True      4
dtype: int64

Inspect Carmelina Tusa fatal crash (139 @ Baldwin, 10/22/21):

In [256]:
tusa_dot_id = 5950657
sp2[sp2.dot_id == tusa_dot_id]

,LOCATION,STREET,cc,cn,dot_id,dt,mc,mn,mp,pk,rt,street,ti,tk
ACCID,,,,,,,,,,,,,,
11309,State Highway 139 (Upper (State Route 139 Only...,None,9,Hudson,5950657,2021-10-22 07:10:00,6,Jersey City,0.58,1,139,State Highway 139 (Upper (State Route 139 Only)),0,1


In [257]:
dot.loc[[tusa_dot_id]]

,year,cc,mc,case,cn,mn,pdc,pdn,station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event,ocn,reason,ilon,ilat,icn
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5950657,2021,9,6,21T9166,Hudson,Jersey City,04,Port Authority Of Ny And,PATH,0,1,0,1,i,I,False,False,13,1,NJ 139 LOWER,E,139,,00000139__,0.86,02,,00,01,01,01,00,,,AT,,BALDWIN AVE,,,,,,NaN,NaN,N,,2538,2021-10-22 07:10:00,00,00,22,None,None,-74.054562,40.733341,Hudson


Compare roads for crashes recovered in round 2 (`['dt','cc']`):

In [262]:
def cmp_roads(sp):
    return (
        sp
        .reset_index()
        [[ 'ACCID', 'dot_id', 'street', 'mp', 'cn', 'mc', 'mn', ]]
        .merge(
            dotr[['dot_id', 'Crash Location', 'Cross Street Name', 'sri', 'mp', 'cn', 'mc', 'mn', 'ilat', 'ilon', 'olat', 'olon', ]],
            how='left',
            on='dot_id',
            suffixes=['_sp', '_dot'],
        )
        .set_index('ACCID')
        .sort_index()
        .rename(columns={
            'street': 'street_sp',
            'Crash Location': 'street_dot',
            'Cross Street Name': 'cross_dot',
        })
        [[ 'dot_id', 'cn_sp', 'cn_dot', 'mc_sp', 'mc_dot', 'mn_sp', 'mn_dot', 'street_sp', 'street_dot', 'cross_dot', 'mp_sp', 'mp_dot', 'sri', 'ilat', 'ilon', 'olat', 'olon', ]]
    )

In [263]:
st2 = cmp_roads(sp2)
st2

,dot_id,cn_sp,cn_dot,mc_sp,mc_dot,mn_sp,mn_dot,street_sp,street_dot,cross_dot,mp_sp,mp_dot,sri,ilat,ilon,olat,olon
ACCID,,,,,,,,,,,,,,,,,
10630,6031342,Ocean,Ocean,33,33,Barnegat Twsp,Barnegat Twp,Garden State Parkway,GARDEN STATE PARKWAY,,68.10,68.10,00000444__,39.763231,-74.249111,39.763210,-74.249020
10721,6039704,Passaic,Passaic,8,8,Paterson City,Paterson City,East 18th St,PASSAIC COUNTY 653,ELLISON PL / ELLISON ST,NaN,0.31,16000653__,40.917029,-74.156066,NaN,NaN
10778,6039774,Passaic,Passaic,8,8,Paterson City,Paterson City,State Highway 20,NJ 20,17TH AVE,1.30,1.32,00000020__,40.910886,-74.133871,NaN,NaN
10785,6019832,Ocean,Ocean,7,7,Toms River,Toms River Twp,State Highway 37,NJ 37,DOUGLAS CT,11.95,11.95,00000037__,39.946761,-74.104899,39.949360,-74.114910
11034,5908836,Cumberland,Cumberland,14,14,Vineland City,Vineland City,County 673,CUMBERLAND COUNTY 673,PANTHER RD,4.50,4.97,06000673__,39.450654,-74.948076,39.450578,-74.946405
11230,5854250,Atlantic,Atlantic,12,12,Hamilton Twsp,Hamilton Twp,County 614,ATLANTIC COUNTY 614,US 322 / BLACK HORSE PIKE,0.90,0.94,01000614__,39.458861,-74.660871,39.457883,-74.661969
11234,6045864,Salem,Salem,13,13,Carneys Point Twsp,Carneys Point Twp,State Highway 140,NJ 140,US 130 / SHELL RD,0.30,0.27,00000140__,39.684958,-75.483795,39.684997,-75.483919
11309,5950657,Hudson,Hudson,6,6,Jersey City,Jersey City,State Highway 139 (Upper (State Route 139 Only)),NJ 139 LOWER,BALDWIN AVE,0.58,0.86,00000139__,40.733341,-74.054562,NaN,NaN
11340,6056263,Sussex,Sussex,22,22,Vernon Twsp,Vernon Twp,County 515,ROUTE 515,NJ 94,18.20,8.55,00000515__,41.198602,-74.482896,NaN,NaN


In [264]:
st3 = cmp_roads(sp3)
st3

,dot_id,cn_sp,cn_dot,mc_sp,mc_dot,mn_sp,mn_dot,street_sp,street_dot,cross_dot,mp_sp,mp_dot,sri,ilat,ilon,olat,olon
ACCID,,,,,,,,,,,,,,,,,
10658,5884929,Burlington,Burlington,20,19,Maple Shade Twsp,Maple Shade Twp,State Highway 73,NJ 73,CR 610 / FELLOWSHIP RD,29.02,29.05,00000073__,39.948029,-74.985312,NaN,NaN
10675,5885393,Burlington,Burlington,21,20,Medford Twsp,Medford Twp,County 623,BURLINGTON COUNTY 623,WHITE PINE DR,2.04,2.04,03000623__,39.878581,-74.836972,NaN,NaN
10696,5893365,Camden,Camden,12,9,Cherry Hill Twsp,Cherry Hill Twp,State Highway 70,NJ 70,FRONTAGE RD / OLD CUTHBERT RD,5.20,5.25,00000070__,39.907925,-74.981076,NaN,NaN
10718,5969312,Mercer,Mercer,10,14,Princeton,Princeton,Ewing St,HARRISON ST / N HARRISON ST,MT LUCAS RD,NaN,1.61,11142239__,40.369922,-74.658826,NaN,NaN
10727,6044863,Salem,Salem,7,6,Oldmans Twsp,Oldmans Twp,Interstate 295,I-295,,7.60,7.60,00000295__,39.736256,-75.399317,39.739150,-75.395400
10740,5889864,Burlington,Burlington,38,37,Westampton Twsp,Westampton Twp,County 626,BURLINGTON COUNTY 626,NEW JERSEY TURNPIKE,6.88,6.88,03000626__,40.009133,-74.839910,40.009190,-74.840140
10788,5886766,Burlington,Burlington,24,23,Mount Holly Twsp,Mount Holly Twp,State Highway 38,NJ 38,CR 612 / EAYRESTOWN RD / PINE ST,16.49,16.49,00000038__,39.982263,-74.783234,39.982400,-74.783400
10802,5899049,Camden,Camden,4,18,Bellmawr Boro,Haddon Heights Boro,Interstate 295,I-295,,28.40,28.40,00000295__,39.872588,-75.077674,39.872500,-75.077430
10808,5893638,Camden,Camden,12,9,Cherry Hill Twsp,Cherry Hill Twp,State Highway 38,NJ 38,LONGWOOD AVE,2.00,1.77,00000038__,39.935980,-75.049686,NaN,NaN


## fsck county codes/names

In [276]:
sp_cc_hist = sp[['cn', 'cc']].value_counts().sort_index() # reset_index(level=1)
assert sp_cc_hist.reset_index().cn.value_counts().value_counts().index.tolist() == [1]
sp_cc_hist

cn          cc
Atlantic    1     46
Bergen      2     35
Burlington  3     40
Camden      4     67
Cape May    5     12
Cumberland  6     27
Essex       7     60
Gloucester  8     42
Hudson      9     29
Hunterdon   10     9
Mercer      11    35
Middlesex   12    45
Monmouth    13    32
Morris      14    25
Ocean       15    43
Passaic     16    24
Salem       17    20
Somerset    18    25
Sussex      19    11
Union       20    27
Warren      21    13
dtype: int64

In [275]:
dot_cc_hist = dotr[['cn', 'cc']].value_counts().sort_index()
assert dot_cc_hist.reset_index().cn.value_counts().value_counts().index.tolist() == [1]
dot_cc_hist

cn          cc
Atlantic    1      5773
Bergen      2     25062
Burlington  3      9560
Camden      4     12905
Cape May    5      2309
Cumberland  6      3928
Essex       7     25156
Gloucester  8      7079
Hudson      9     16368
Hunterdon   10     2811
Mercer      11     8469
Middlesex   12    21823
Monmouth    13    15053
Morris      14    10788
Ocean       15    13937
Passaic     16    13249
Salem       17     1574
Somerset    18     7941
Sussex      19     2378
Union       20    18068
Warren      21     2727
dtype: int64

In [307]:
cc_hist = sxs(
    sp_cc_hist.rename('sp_count'),
    dot_cc_hist.rename('dot_count'),
)
assert len(cc_hist) == len(sp_cc_hist)
cc_hist

,,sp_count,dot_count
cn,cc,,
Atlantic,1,46,5773
Bergen,2,35,25062
Burlington,3,40,9560
Camden,4,67,12905
Cape May,5,12,2309
Cumberland,6,27,3928
Essex,7,60,25156
Gloucester,8,42,7079
Hudson,9,29,16368


In [298]:
mns = sxs(sp_ccmc2mn.rename('sp_mn'), dot_ccmc2mn.rename('dot_mn'))
mns

sp_mn           dot_mn
cc mc                                  
1  1      Absecon City     Absecon City
   2     Atlantic City    Atlantic City
   3   Brigantine City  Brigantine City
   5   Buena Vista Twp  Buena Vista Twp
   8    Egg Harbor Twp   Egg Harbor Twp
...                ...              ...
21 14              NaN      Liberty Twp
   15              NaN    Lopatcong Twp
   16              NaN    Mansfield Twp
   20              NaN    Pohatcong Twp
   21              NaN  Washington Boro

[557 rows x 2 columns]

In [304]:
mns[mns.dot_mn == 'Princeton']

,,sp_mn,dot_mn
cc,mc,,
11,14,NaN,Princeton


In [303]:
mns[~mns.sp_mn.isna() & mns.dot_mn.isna()]

,,sp_mn,dot_mn
cc,mc,,
11,10,Princeton,NaN


In [302]:
pd.crosstab(mns.sp_mn.isna(), mns.dot_mn.isna())

dot_mn,False,True
sp_mn,,
False,243,1
True,313,0


---

In [235]:
from nj_crashes.sri import SRI

In [236]:
passaic653 = SRI('16000653__')
passaic653

SRI(sri='16000653__', _mp2ll=None)

In [237]:
p653_df = passaic653.features_df
p653_df

,OBJECTID_1,SRI,MP,ROUTE_SUBT,SLD_NAME,Second_Name,x,y
0,650991,16000653__,0.00,6,PASSAIC COUNTY 653,None,-74.157453,40.912605
1,874228,16000653__,0.05,6,PASSAIC COUNTY 653,None,-74.157201,40.913378
2,649825,16000653__,0.10,6,PASSAIC COUNTY 653,None,-74.156978,40.914097
3,876243,16000653__,0.15,6,PASSAIC COUNTY 653,None,-74.156764,40.914788
4,650749,16000653__,0.20,6,PASSAIC COUNTY 653,None,-74.156548,40.915483
5,872269,16000653__,0.25,6,PASSAIC COUNTY 653,None,-74.156326,40.916188
6,650283,16000653__,0.30,6,PASSAIC COUNTY 653,None,-74.156106,40.916890
7,872210,16000653__,0.35,6,PASSAIC COUNTY 653,None,-74.155905,40.917581
8,650099,16000653__,0.40,6,PASSAIC COUNTY 653,None,-74.155679,40.918287
9,875040,16000653__,0.45,6,PASSAIC COUNTY 653,None,-74.155444,40.918977


In [231]:
sp.loc[10721]

CNAME                   Passaic
MNAME             Paterson City
rt                             
LOCATION           East 18th St
tk                            1
ti                            0
STREET             East 18th St
pk                            1
dt          2021-02-13 22:39:00
cc                           16
mc                            8
street             East 18th St
mp                          NaN
Name: 10721, dtype: object

In [217]:
dot.loc[6039704]

year                                                 2021
cc                                                     16
mc                                                      8
case                                        I-2021-013413
cn                                                Passaic
mn                                          Paterson City
pdc                                                    01
pdn                                           Paterson Pd
station                                      111 BROADWAY
Total Killed                                            0
Total Injured                                           1
Pedestrians Killed                                      0
Pedestrians Injured                                     1
severity                                                i
Intersection                                            B
Alcohol Involved                                    False
HazMat Involved                                     False
crash_type    

In [192]:
sp1[['dot_id']].merge(dotr[['dot_id', 'severity']], on='dot_id').severity.value_counts()

f    534
Name: severity, dtype: int64

Round 2: NJSP fatal crashes listed as non-fatal by DOT:

In [193]:
m2 = sp2[['dot_id']].merge(dotr, on='dot_id')
m2[m2.severity != 'f']

,dot_id,year,cc,mc,case,cn,mn,pdc,pdn,station,tk,ti,pk,Pedestrians Injured,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,Total Vehicles Involved,Crash Location,Location Direction,rt,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event,ocn,reason,ilon,ilat,icn
5,6039704,2021,16,8,I-2021-013413,Passaic,Paterson City,01,Paterson Pd,111 BROADWAY,0,1,0,1,i,B,False,False,13,1,PASSAIC COUNTY 653,N,653,,16000653__,0.31,05,,02,02,06,01,05,01,20,FE,N,ELLISON PL / ELLISON ST,,,,25,25,NaN,NaN,N,,4972,2021-02-13 22:39:00,01,04,22,None,None,-74.156066,40.917029,Passaic
8,6039774,2021,16,8,I-2021-016208,Passaic,Paterson City,01,Paterson Pd,PATROL/A1/103,0,4,0,0,i,I,False,False,04,3,NJ 20,S,20,,00000020__,1.32,02,,02,01,02,05,04,01,,AT,E,17TH AVE,,,,45,25,NaN,NaN,N,,4852,2021-02-24 04:51:00,01,04,26,None,None,-74.133871,40.910886,Passaic
9,6019832,2021,15,7,21-09990,Ocean,Toms River Twp,01,Toms River Pd,DISTRICT ONE,0,2,0,0,i,B,False,False,02,2,NJ 37,E,37,,00000037__,11.95,02,,01,01,01,01,05,01,3000,FE,E,DOUGLAS CT,,,,40,25,39.949360,-74.114910,N,SEVERAL SECTIONS OF NJ DOT MATTHIS BRIDGE GUAR...,0399,2021-03-15 11:02:00,01,04,26,Ocean,None,-74.104899,39.946761,Ocean
36,5908836,2021,6,14,2021032391,Cumberland,Vineland City,01,Vineland Pd,VINELAND PD,0,1,0,1,i,B,False,False,13,1,CUMBERLAND COUNTY 673,W,673,,06000673__,4.97,05,,02,01,07,01,05,01,500,FE,W,PANTHER RD,,,,50,45,39.450578,-74.946405,N,,7578,2021-07-09 23:22:00,01,04,22,Cumberland,None,-74.948076,39.450654,Cumberland
47,5854250,2021,1,12,2021034773,Atlantic,Hamilton Twp,01,Hamilton Twp Pd (Atlantic,MAYS LANDING,0,1,0,0,i,B,False,False,14,1,ATLANTIC COUNTY 614,N,614,,01000614__,0.94,05,,02,01,01,01,05,01,500,FE,N,US 322 / BLACK HORSE PIKE,,,,50,50,39.457883,-74.661969,N,,0067,2021-09-25 14:02:00,01,04,21,Atlantic,None,-74.660871,39.458861,Atlantic
48,6045864,2021,17,13,CP-2021-08172,Salem,Carneys Point Twp,01,Carneys Point Pd,MUNICIPAL,0,2,0,0,i,B,False,False,07,2,NJ 140,,140,,00000140__,0.27,02,,02,01,01,01,05,01,1430,FE,E,US 130 / SHELL RD,,,,35,45,39.684997,-75.483919,N,,1183,2021-09-19 12:02:00,01,04,26,Salem,None,-75.483795,39.684958,Salem
50,5950657,2021,9,6,21T9166,Hudson,Jersey City,04,Port Authority Of Ny And,PATH,0,1,0,1,i,I,False,False,13,1,NJ 139 LOWER,E,139,,00000139__,0.86,02,,00,01,01,01,00,,,AT,,BALDWIN AVE,,,,,,NaN,NaN,N,,2538,2021-10-22 07:10:00,00,00,22,None,None,-74.054562,40.733341,Hudson
53,6056263,2021,19,22,21-21194,Sussex,Vernon Twp,01,Vernon Twp Pd,,0,1,0,1,i,B,False,False,13,1,ROUTE 515,E,515,,00000515__,8.55,05,,02,02,03,04,05,01,150,FE,E,NJ 94,,,,35,45,NaN,NaN,N,,74,2021-10-30 17:14:00,01,06,22,None,None,-74.482896,41.198602,Sussex
55,6001736,2021,13,34,21NT26477,Monmouth,Neptune Twp,01,Neptune Twp Pd,,0,6,0,0,i,I,False,False,03,3,NJ 35,S,35,,00000035__,23.93,02,,02,01,01,01,05,01,,AT,,WEST LAKE AVE,,,,35,25,40.215780,-74.030540,N,,34205,2021-11-03 13:56:00,01,04,26,Monmouth,None,-74.030545,40.215810,Monmouth
58,5949586,2021,9,6,21-230795,Hudson,Jersey City,01,Jersey City Pd,WEST,0,1,0,1,i,B,False,False,13,1,ROUTE 501,,501,,00000501__,29.45,05,,01,01,06,01,04,01,100,FE,N,HARRISON AVE,,,,25,25,NaN,NaN,N,,2548,2021-11-17 19:26:00,01,04,22,None,None,-74.076978,40.720380,Hudson


In [154]:
dotk.loc[list(multi_groups.to_dict().keys())[0]]

/var/folders/dc/gqj_cd8d0d9c1nppd5_yl7bc0000gn/T/ipykernel_1334/4019040234.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  dotk.loc[list(multi_groups.to_dict().keys())[0]]


dt          cc  mc  tk
2021-01-18  14  35  0     6015684
                    0     6015685
Name: dot_id, dtype: int64

In [161]:
dot_ids = dotr.set_index(keys).dot_id.sort_index()
dot_dict = dot_ids.to_dict()

In [163]:
%%time
nxt = (
    sp
    .iloc[:100]
    .reset_index()
    .apply(match_crash, keys=keys, dot_ids=dot_ids, dot_dict=dot_dict, axis=1)
)
nxt

CPU times: user 84.3 ms, sys: 10.7 ms, total: 94.9 ms
Wall time: 96.4 ms


ACCID 10626: dot_id 6071677
ACCID 10627: dot_id 5968575
ACCID 10628: dot_id 5936843
ACCID 10649: dot_id 6010246
ACCID 10651: dot_id 5901291
ACCID 10637: dot_id 5965208
ACCID 10632: dot_id 5987570
ACCID 10633: dot_id 5892434
ACCID 10636: dot_id 5852735
ACCID 10635: dot_id 5851107
ACCID 10641: dot_id 5936295
ACCID 10643: dot_id 6010916
ACCID 10644: dot_id 5850350
ACCID 10648: dot_id 5935165
ACCID 10662: dot_id 5938790
ACCID 10656: dot_id 6009242
ACCID 10657: dot_id 5850629
ACCID 10659: dot_id 5940967
ACCID 10660: dot_id 6045382
ACCID 10663: dot_id 5920863
ACCID 10676: dot_id 5918336
ACCID 10670: dot_id 5884030
ACCID 10667: dot_id 6008506
ACCID 10666: dot_id 5997231
ACCID 10665: dot_id 6007707
ACCID 10673: dot_id 6028418
ACCID 10691: dot_id 5921690
ACCID 10678: dot_id 5993696
ACCID 10674: dot_id 6022316
ACCID 10695: dot_id 5856264
ACCID 10773: dot_id 5944967
ACCID 10679: dot_id 6046088
ACCID 10719: dot_id 5909948
ACCID 10680: dot_id 6014295
ACCID 10692: dot_id 5892505
ACCID 10682: dot_id 

,ACCID,CNAME,LOCATION,MNAME,STREET,cc,dot_id,dt,mc,mp,pk,rt,street,ti,tk
0,10630,Ocean,Garden State Parkway MP 68.1,Barnegat Twsp,None,15,NaN,2021-01-01 07:45:00,33,68.10,0,444,Garden State Parkway,0,1
1,10626,Union,Interstate 78 MP 50.8,Union Twsp,None,20,6071677.0,2021-01-01 09:06:00,19,50.80,0,78,Interstate 78,0,1
2,10627,Mercer,State Highway 1 MP 11.83,West Windsor Twp,None,11,5968575.0,2021-01-02 09:07:00,13,11.83,0,1,State Highway 1,1,1
3,10628,Gloucester,County 555 MP 24.9,Franklin Twsp,None,8,5936843.0,2021-01-03 04:19:00,5,24.90,0,555,County 555,2,1
4,10649,Morris,State Highway 46 MP 50.5,Montville Twsp,None,14,6010246.0,2021-01-03 15:37:00,21,50.50,0,46,State Highway 46,0,1
5,10651,Camden,State Highway 168 MP 5.78,Runnemede Boro,None,4,5901291.0,2021-01-03 16:41:00,30,5.78,1,168,State Highway 168,0,1
6,10637,Mercer,Genesee St,Trenton,Genesee St,11,5965208.0,2021-01-03 21:07:00,11,NaN,1,,Genesee St,0,1
7,10632,Middlesex,County 616 MP .29,Woodbridge Twsp,None,12,5987570.0,2021-01-04 13:35:00,25,NaN,1,616,County 616 MP .29,2,1
8,10633,Camden,Fairview St,Camden City,Fairview St,4,5892434.0,2021-01-05 02:03:00,8,NaN,0,,Fairview St,2,1
9,10636,Atlantic,Garden State Parkway MP 37.4,Egg Harbor Twsp,None,1,5852735.0,2021-01-06 14:28:00,8,37.40,0,444,Garden State Parkway,0,1


In [152]:
idx_groups = dotk.groupby(lambda _:_).count().sort_values()
multi_groups = idx_groups[idx_groups > 1]
multi_groups

(2021-01-18 00:00:00, 14, 35, 0)     2
(2021-12-19 05:30:00, 3, 24, 0)      2
(2021-06-11 23:33:00, 13, 28, 0)     2
(2021-09-28 20:39:00, 7, 14, 0)      2
(2021-04-30 19:18:00, 20, 12, 0)     2
                                    ..
(2021-04-22 23:46:00, 8, 9, 0)       3
(2021-09-09 18:19:00, 3, 37, 0)      4
(2021-11-26 06:31:00, 13, 18, 0)     4
(2021-02-08 19:00:00, 16, 7, 0)      6
(2021-02-13 17:37:00, 3, 1, 0)      11
Name: dot_id, Length: 1026, dtype: int64

In [96]:
sp1 = (
    sp
    .reset_index()
    .merge(
        dotr,
        left_on=['dt', 'cc', 'mc', 'tk'],
        right_on=['dt', 'cc', 'mc', 'tk'],
        how='left', 
#         validate='one_to_one',
    )
    .set_index('ACCID')
)
sp1

,CNAME,MNAME,rt_x,LOCATION,tk,ti_x,STREET,pk_x,dt,cc,mc,street,mp_x,dot_id,year,case,cn,mn,pdc,pdn,station,ti_y,pk_y,Pedestrians Injured,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,Total Vehicles Involved,Crash Location,Location Direction,rt_y,Route Suffix,sri,mp_y,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,Road Horizontal Alignment,Road Grade,First Harmful Event,ocn,reason,ilon,ilat,icn
ACCID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10630,Ocean,Barnegat Twsp,444,Garden State Parkway MP 68.1,1,0,None,0,2021-01-01 07:45:00,15,33,Garden State Parkway,68.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10626,Union,Union Twsp,78,Interstate 78 MP 50.8,1,0,None,0,2021-01-01 09:06:00,20,19,Interstate 78,50.80,6071677.0,2021.0,B130-2021-00003A,Union,Union Twp,02,New Jersey State Police,SOMERVILLE - SQ,0.0,0.0,0.0,f,B,True,False,11,1.0,I-78,E,78,,00000078__,50.80,01,,02,01,01,01,01,01,,,,,,,,65,,40.714110,-74.295980,N,Approximately 50 feet of guardrail damage. NJ...,7116,01,04,46,Union,None,-74.293528,40.713984,Union
10627,Mercer,West Windsor Twp,1,State Highway 1 MP 11.83,1,1,None,0,2021-01-02 09:07:00,11,13,State Highway 1,11.83,5968575.0,2021.0,2021-104,Mercer,West Windsor Twp,01,West Windsor Twp Pd,2021-104,1.0,0.0,0.0,f,I,False,False,03,2.0,US 1,N,1,,00000001__,11.83,02,,02,02,01,01,01,01,,AT,,CR 629 / HARRISON ST,,,,55,35,NaN,NaN,N,,,01,04,26,None,None,-74.630920,40.337309,Mercer
10628,Gloucester,Franklin Twsp,555,County 555 MP 24.9,1,2,None,0,2021-01-03 04:19:00,8,5,County 555,24.90,5936843.0,2021.0,2021000110,Gloucester,Franklin Twp,01,Franklin Twp Pd (Gloucest,PATROL,2.0,0.0,0.0,f,B,True,False,11,1.0,ROUTE 555,S,555,,00000555__,24.90,05,,02,01,07,05,05,01,1557,FE,N,MARSHALL MILL RD,,,,50,45,39.601995,-75.006492,N,MAILBOX OF 3172 MAIN ROAD: (ROBERT DEVENEY 10...,1174,01,05,26,Gloucester,None,-75.006558,39.601873,Gloucester
10649,Morris,Montville Twsp,46,State Highway 46 MP 50.5,1,0,None,0,2021-01-03 15:37:00,14,21,State Highway 46,50.50,6010246.0,2021.0,2021000094,Morris,Montville Twp,01,Montville Twp Pd,MONTVILLE TOWN?,0.0,0.0,0.0,f,B,False,False,11,1.0,US 46,E,46,,00000046__,50.54,02,,02,02,01,07,01,01,900,FE,W,HOOK MTN RD,,,,50,35,40.858873,-74.343984,N,THE PINE BROOK MOTEL BUILDING (CURRENTLY UNDER...,083,01,04,48,Morris,None,-74.344339,40.858325,Morris
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11456,Essex,Newark City,95,New Jersey Turnpike MP 105.5,1,3,None,1,2021-12-29 04:33:00,7,14,New Jersey Turnpike,105.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11457,Camden,Winslow Twsp,446,Atlantic City Expressway MP 39.4,1,0,None,0,2021-12-30 01:46:00,4,36,Atlantic City Expressway,39.40,5903568.0,2021.0,210683085,Camden,Winslow Twp,02,New Jersey State Police,A160,0.0,0.0,0.0,f,B,False,False,12,2.0,ATLANTIC CITY EXPRESSWAY,W,446,,00000446__,39.40,03,,02,02,05,02,03,01,,,,,,,,65,,39.716500,-74.990600,N,,8104,01,04,24,Camden,None,-74.996803,39.721532,Camden
11470,Ocean,Lakewood Twsp,623,County 623 MP 6.11,1,1,None,0,2021-12-30 06:31:00,15,14,County 623,6.11,6027864.0,202

In [97]:
sp1_mask = sp1.dot_id.isna()
sp1_mask.value_counts()

False    534
True     133
Name: dot_id, dtype: int64

In [18]:
sm1 = sp.set_index('ACCID')[sp1_mask]
sm1

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,INJURIES,STREET,FATAL_D,FATAL_P,FATAL_T,FATAL_B,dt
ACCID,,,,,,,,,,,,,,
10630,15,Ocean,1533,Barnegat Twsp,444,Garden State Parkway MP 68.1,1.0,0.0,None,1.0,0.0,0.0,0.0,2021-01-01 07:45:00
10668,08,Gloucester,0809,Logan Twsp,295,Interstate 295 MP 15.1,1.0,0.0,None,1.0,0.0,0.0,0.0,2021-01-19 12:32:00
10694,04,Camden,0414,Gloucester City,130,State Highway 130 MP 26.9,1.0,2.0,None,1.0,0.0,0.0,0.0,2021-01-25 05:03:00
10721,16,Passaic,1608,Paterson City,None,East 18th St,1.0,0.0,East 18th St,0.0,0.0,1.0,0.0,2021-02-13 22:39:00
10778,16,Passaic,1608,Paterson City,20,State Highway 20 MP 1.3,1.0,3.0,None,0.0,1.0,0.0,0.0,2021-02-24 04:51:00
10733,09,Hudson,0906,Jersey City,None,Christopher Columbus Dr,1.0,1.0,Christopher Columbus Dr,0.0,1.0,0.0,0.0,2021-02-28 03:28:00
10755,12,Middlesex,1219,Sayreville Boro,35,State Highway 35 MP 47.43,1.0,0.0,None,1.0,0.0,0.0,0.0,2021-02-28 15:21:00
10764,09,Hudson,0906,Jersey City,None,John F. Kennedey Blvd,1.0,0.0,John F. Kennedey Blvd,1.0,0.0,0.0,0.0,2021-03-10 12:16:00
10785,15,Ocean,1507,Toms River,37,State Highway 37 MP 11.95,1.0,1.0,None,1.0,0.0,0.0,0.0,2021-03-15 11:02:00


In [32]:
(sp1.MCODE.str[:2] == sp1.CCODE).all()

True

In [31]:
dotr.apply(lambda r: '%02d%02d' % (r.cc, r.mc), axis=1)

0         1016
1         0101
2         0318
3         2004
4         1608
          ... 
226953    0906
226954    0906
226955    1111
226956    0504
226957    0404
Length: 226958, dtype: object

In [19]:
len(sm1)

76

In [25]:
sp.MCODE.value_counts()

0714    36
0408    19
0906    15
1111    12
0436    11
        ..
0609     1
1303     1
1513     1
1804     1
0411     1
Name: MCODE, Length: 244, dtype: int64

In [26]:
dotr[['cc', 'mc']].value_counts()

cc  mc
7   14    12325
9   6      6210
20  4      5214
12  25     4038
15  14     3949
          ...  
4   37        2
14  30        1
17  3         1
13  20        1
3   14        1
Length: 556, dtype: int64

In [24]:
dotr[['dt', 'cn']].value_counts().value_counts()

1     215992
2       5226
3        159
4          5
11         1
6          1
dtype: int64

In [23]:
dotr[['dt', 'cn', 'mn']].value_counts().value_counts()

1     224856
2       1022
3         11
4          2
11         1
6          1
dtype: int64

In [21]:
sp2 = (
    sm1
    .merge(
        dotr,
        left_on=['dt', 'CNAME'],
        right_on=['dt', 'cn'],
        how='left', 
        validate='one_to_one',
    )
    .set_index('ACCID')
)
sp2

MergeError: Merge keys are not unique in right dataset; not a one-to-one merge